In [34]:


#!pip install numpy 

import os 
import numpy as np 
import pandas as pd 
from numpy.linalg import norm
import re 



class StringSimilarity: 
    
    """
    A class for computing string similarity using various metrics.
    This class provides functionality to clean and process text documents,
    calculate similarity scores, and manage a collection of text documents.
    
    """
    
    def __init__(self): 
        
        """
        Initializes the StringSimilarity class with empty structures for storing documents.
        
        """
        
        # Dictionary to store the processed documents
        self.document_pool = {}
        
        # Dictionary to store vector representations of the documents
        self.vector_pool = {}
        
        # Set to store unique words across all documents
        self.dictionary = set()
        
        
        # list of stopwords for basic text filtering
        self.stopwords = [
            
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", 
        "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", 
        "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", 
        "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", 
        "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", 
        "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", 
        "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", 
        "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", 
        "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
        "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", 
        "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", 
        "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now" 
        ]

    
    
    
    def add_documents(self,name, document): 
        
        """
        Manual adds a document to the document pool after processing it.

        Args:
            name (str): The name or identifier for the document.
            document (str): The text of the document to be added.

        Raises:
            ValueError: If document or name is string
            ValueError: If name or document is empty 
            ValueError: Processed document is empty. It might contain only stopwords or non-words
            ValueError: If the processed document already exists in the document pool.
        """
        
        
        if not isinstance(name, str) or not isinstance(document, str):
            
            
            raise TypeError("Both name and document must be strings.")
        
        if not name:
            raise ValueError("Document name is empty.")
    
        if not document:
            raise ValueError("Document content is empty.")
        
        
        processed_document = self.main_cleaning(document)
        
        if not processed_document:
            raise ValueError("Processed document is empty. It might contain only stopwords or non-words.")
        
        
        # Check if the document is not already in the pool
        if processed_document not in list(self.document_pool.keys()): 
            
            self.document_pool[name] = processed_document
            
            if not bool(set(processed_document) & self.dictionary): 
            
                self.dictionary.update(set(processed_document))
                
                # after a new document is added to pool, all vectors have to be updated because dictionary is longer. 
                self.update_vectorpool() 
            else: 
                self.vector_pool[name] = self.create_vector(processed_document)
                
                print("Text has been added to the pool but no new vocabularies were added.")
            
        else: 
            raise ValueError(f"The text {processed_document} has already been added to pool")
    


    
    @staticmethod
    def cleaning_text(text): 
        
        
        """
        Static method to clean a given text.

        Args:
            text (str): The text to be cleaned.

        Returns:
            str: The cleaned text.
            
        Raises: 
            TypeError: If the input text is not a string.
            ValueError: If Input text is empty or only contains whitespace
            
        """
        
        if not isinstance(text, str):
            raise TypeError("Input text must be a string.")
        
        if text.strip() == "":
            
            raise ValueError("Input text is empty or only contains whitespace.")
        
        
        text = text.strip() # removes whitespaces in the beginning and end
        text = re.sub(r'\b[_-]+|(?<=\w)[_-]+|[_-]+(?=\w)', '', text) # Removes hyphens or underscores that are surrounded by word characters.
        text = re.sub(r'\b(?:[a-zA-Z]\.)+[a-zA-Z]?[,]*\b', ' ', text) # Replaces abbreviations or initials and optional trailing commas with a space.
        text = re.sub(r"\W", " ", text)  #remove non words char
        text = re.sub(r"\d", " ", text)  #remove digits char
        text = re.sub(r"[\s]+", " ", text) # remove extra white space
        text = text.lower() #lower char for matching
        
        return text 

    
    @staticmethod
    def load_text(path):
        
        """
        Static method to load text from a given file path.

        Args:
            path (str): The file path from which to load the text.

        Returns:
            list: The processed list of words from the file.
        
        Raises: 
            ValueError: If input is not a string
            FileNotFoundError: If the path can not be found within the operating system 
        """
        if not isinstance(path, str): 
            
            raise ValueError("The file path must be a string.")
        
        if not os.path.exists(path):
            raise FileNotFoundError(f"The file does not exist at the path: {path}")
        
        
        # add try ... except???
        with open(path, 'r') as file: #Automatically closes the file after reading
            
            #file = StringSimilarity.string_to_list(file.read())
            
            file = file.read()
        
        return file
    
    
    @staticmethod
    def create_doc_list(curr_path): 
        
        """
        Static method to create a list of document names in the 'Corpus' directory.

        Args:
            curr_path (str): The current working directory path.

        Returns:
            list: A list of filenames found in the 'Corpus' subdirectory.
            
        Raises: 
            FileNotFoundError: If the path can not be found within the operating system 
        """
        
        # Construct the path to the 'Corpus' directory which contains .txt files
        corpus_path = os.path.join(curr_path, 'Corpus')
        
        if not os.path.exists(corpus_path):
            raise FileNotFoundError(f"The file does not exist at the path: {corpus_path}")

        # List all files in the 'Corpus' directory
        objects = os.listdir(corpus_path)
        
        return objects 

    def create_corpus(self): 
        
        """
        Method to create a corpus by processing and adding text files from the 'Corpus' directory.
        Updates the document pool with new documents and their processed content.
        
        Returns:
            str: A message indicating the outcome of the corpus creation
        
        Raises: 
            Exception: If an unexpected error occurs during file processing.
        
        """
        
        # Get the current working directory
        path = os.getcwd()
        
        # Retrieve the list of text files in the 'Corpus' directory
        
        try: 
            
            text_files = StringSimilarity.create_doc_list(path)
            
        except Exception as e: 
            
            raise Exception(f'Failed to create document list: {e}')
        
        # create path to Corpus folder 
        corpus_path = os.path.join(path, 'Corpus')
        
        # count number of documents
        new_count = 0
        
        
        
        for i in text_files: 
            
            # Process only text files and avoid duplicates
            if i.endswith('.txt'): 
                
                # avoid duplicates in document pool
                if i not in self.document_pool.keys(): 
                    
                    
                    try: 
                    # Load and process the text file
                        temp_text = StringSimilarity.load_text(os.path.join(corpus_path, i))
                        
                        
                        temp_text = self.main_cleaning(temp_text)
            
                        
                        
                    

                        # Update the dictionary and document pool
                        self.dictionary.update(set(temp_text))
                        self.document_pool[i] = list(set(temp_text))
                        new_count+= 1 
                    except Exception as e: 
                        
                        raise Exception(f'Failed to load document {i} because of {e}')
                        
                    
                else: 
                    continue
            else: 
                continue 
        
        # Update the vector pool with new vectors
        
        
        self.update_vectorpool()     
        
        if new_count == 0: 
            
            return "no new documents in folder"
        else: 
            
            return f"where have been {str(new_count)} new documents in the folder"
            
    
    
    
    @staticmethod
    def string_to_list(string1): 
        
        """
        Static method to convert a cleaned string into a list of words.

        Args:
            string1 (str): The string to be converted.

        Returns:
            list: A list of words from the string.
            
        Raises: 
            TypeError: If string is not string
            ValueError: If string is empty after cleaning 
        """
        
        if not isinstance(string1, str):
            raise TypeError("Input must be a string.")
        
        
        # Convert the cleaned string into a list of words
        clean_text = StringSimilarity.cleaning_text(string1)

        if not clean_text.strip():
            raise ValueError("Input string is empty or contains only whitespace after cleaning.")
        
        return clean_text.split()


    def removing_stopwords(self, list_words): 
        
        """
        Method to remove stopwords from a list of words.

        Args:
            list_words (list): The list of words from which stopwords are to be removed.

        Returns:
            list: A list of words with stopwords removed.
            
        Raises: 
            TypError: If Type of Input is not a list of words 
            ValueError: If list from Input is empty

            
        """
        if not isinstance(list_words, list):
            raise TypeError("Input must be a list of words.")

        if not list_words:
            raise ValueError("Input list of words is empty.")
        
        
        # Filter out stopwords from the list of words
        text_without_stop = [word for word in list_words if word not in self.stopwords]
        
        return text_without_stop
    
    
    def main_cleaning(self, text): 
        
        """
        Method to perform cleaning of the text, converting it into a list of words and removing stopwords. Finally remove words which are shorter than 3

        Args:
            text (str): The text to be cleaned.

        Returns:
            list: A list of cleaned words from the text.
            
        Raises: 
            TypeError: If input is not a string 
            ValueError: If the input text is empty.
        """
        
        if not isinstance(text, str):
            raise TypeError("Input must be a string.")
        
        
        if text.strip() == "":
            raise ValueError("Input text is empty or only contains whitespace.")
        
        # Clean text, convert text to a list of words and remove stopwords
        text_list = StringSimilarity.string_to_list(text)
        text_list = self.removing_stopwords(text_list)
        
        # Filter out words that are too short (e.g., less than 3 characters)
        text_list = [word for word in text_list if len(word) > 2]
        
        return text_list      
    
   

    def create_vector(self, word_list): 
        
        """
        Creates a binary vector representation for a given list of words.

        Args:
            word_list (list): A list of words to be converted into a vector.

        Returns:
            list: A binary vector where 1 represents the presence of a word from the word list in the dictionary.
            
        Raises: 
            TypeError: If the input is not a list.
            ValueError: If the input list is empty or the dictionary is not initialized.
            
        """

        if not isinstance(word_list, list):
            raise TypeError("Input must be a list of words.")

        if not word_list:
            raise ValueError("Input word list is empty.")

        if not self.dictionary:
            raise ValueError("Dictionary is not initialized. Add some documents first.")
        
        # Initialize a vector of zeros with the same length as the dictionary
        vector = [0] * len(self.dictionary)
        

        # Set elements to 1 in the vector for words present in the word list
        for i, word in enumerate(self.dictionary): 
            
            if word in word_list: 
                vector[i] = 1
            else: 
                continue 
            
        return vector 
    
    
    def update_vectorpool(self):
        
        """
        Updates the vector representations for all documents in the document pool.
        
        Raises: 
            ValueError: If the document pool is empty.
        """ 
        
        
        if not self.document_pool:
            raise ValueError("Document pool is empty. Add some documents before updating the vector pool.")

        # Check if the dictionary is initialized
        if not self.dictionary:
            raise ValueError("Dictionary is not initialized. Add some documents to create the dictionary.")

        try:
            # Update vector for each document in the document pool
            for i in self.document_pool.keys():
                self.vector_pool[i] = self.create_vector(self.document_pool[i])

            print("All vectors are updated")
            
    
        except Exception as e:
            raise Exception(f"An error occurred while updating the vector pool: {e}")
    

    @staticmethod
    def rank_vectors(dict1): 
        
        """
        Ranks vectors based on their values.

        Args:
            dict1 (dict): A dictionary of vectors to be ranked.

        Returns:
            dict: A dictionary with vectors ranked in descending order of their values.
        
        Raises:
            TypeError: If the input is not a dictionary.
            ValueError: If the input dictionary is empty.
        """
        
        
        if not isinstance(dict1, dict):
            raise TypeError("Input must be a dictionary.")

        if not dict1:
            raise ValueError("Input dictionary is empty.")  

        
        # Sort the dictionary in descending order based on values
        return dict(sorted(dict1.items(), key=lambda item: item[1], reverse=True))

        
        
        
    def dot_product_normal(self, new_doc, new_vector): 
        
        """
        Calculates the dot product similarity between a new document and all documents in the document pool.

        Args:
            new_doc (str): The text of the new document.

        Returns:
            dict: A dictionary of dot product similarity scores.
        
        Raises:
            Valueerror: If the type of new_doc is not list.
            ValueError: If the new document list is empty.
            ValueError: If the document pool is empty.
        """
    

        if not isinstance(new_doc, list):
            raise TypeError("The new document must be a list.")

        if len(new_doc) == 0: 
            raise ValueError("The list of words does not contains any words")

        if not self.document_pool:
            raise ValueError("Document pool is empty. Add some documents before calculating Euclidean distance.")
        
        final_dict = {}
        

        
        
        # Calculate dot product with each document vector
        for text in self.document_pool.keys(): 

            final_dict[text] = np.dot(new_vector, self.vector_pool[text])
        
        return StringSimilarity.rank_vectors(final_dict)
    
    

    def cosine_Similarity(self, new_doc,new_vector): 
        
        """
        Calculates the cosine similarity between a new document and all documents in the document pool.

        Args:
            new_doc (str): The text of the new document.

        Returns:
            dict: A dictionary of cosine similarity scores.
            
        Raises:
            ValueError: If the new document is empty or only contains whitespace.
            ValueError: If the document pool is empty.
        """



        if not self.document_pool:
            raise ValueError("Document pool is empty. Add some documents before calculating cosine similarity.")
        
        
        cosine_values = {}
        
        
        # cleans new text and create vector
        # clean_text = self.main_cleaning(new_doc)
        
        # new_vector = self.create_vector(clean_text)
        
        
        # Calculate cosine similarity with each document vector
        for i in self.document_pool.keys(): 
            
            temp_vector = self.vector_pool[i]
            
            if norm(new_vector)*norm(temp_vector) != 0: 
                
                cosine = np.dot(new_vector,temp_vector)/(norm(new_vector)*norm(temp_vector))
                
                cosine_values[i] = cosine
                
            else: 
                cosine_values[i] = 'no matches'
            
        return StringSimilarity.rank_vectors(cosine_values)
    
    
    def Euclidean_distance(self, new_doc, new_vector): 
        
        """
        Calculates the Euclidean distance between a new document and all documents in the document pool.

        Args:
            new_doc (str): The text of the new document.

        Returns:
            dict: A dictionary of Euclidean distance scores.
            
        Raises:
            ValueError: If type of input new_doc is not a list. 
            ValueError: If the new document list is empty.
            ValueError: If the document pool is empty.
            
        """
        if not isinstance(new_doc, list):
            raise TypeError("The new document must be a list.")

        if len(new_doc) == 0: 
            raise ValueError("The list of words does not contains any words")

        if not self.document_pool:
            raise ValueError("Document pool is empty. Add some documents before calculating Euclidean distance.")
        
        euclidean_values = {}
        
        # cleans new text and create vector
        # clean_text = self.main_cleaning(new_doc)
        # new_vector = self.create_vector(clean_text)
        
        
        # Calculate Euclidean distance with each document vector
        for i in self.document_pool.keys(): 
            
            temp_vector = self.vector_pool[i]
            
            dist = np.linalg.norm(np.array(temp_vector) - np.array(new_vector))
            euclidean_values[i] = dist 
            
        return StringSimilarity.rank_vectors(euclidean_values)
    
    def Jaccard_similarity(self, new_doc, clean_words): 
        
        """
        Calculates the Jaccard similarity between a new document and all documents in the document pool.

        Args:
            new_doc (str): The text of the new document.

        Returns:
            dict: A dictionary of Jaccard similarity scores.
            
        Raises:
            TypeError: If the new document is not a list.
            ValueError: If the new document list 
            ValueError: If the document pool is empty.
        """
        if not isinstance(new_doc, list):
            raise TypeError("The new document must be a list.")

        if len(new_doc) == 0: 
            raise ValueError("The list of words does not contains any words")

        if not self.document_pool:
            raise ValueError("Document pool is empty. Add some documents before calculating Jaccard similarity.")
        jaccard_values = {}
        
        # cleans new text and create set of words
        # clean_text = self.main_cleaning(new_doc)
        # set_new_words = set(clean_text)
        
        # Iterate over each document in the document pool
        for name, words in self.document_pool.items(): 
            
            set_old_words = set(words)
            
            # Calculate the intersection and union
            intersection = clean_words.intersection(set_old_words)
            union = clean_words.union(set_old_words)

            # Calculate Jaccard similarity and add to the dictionary
            jaccard_sim = len(intersection) / len(union) if union else 0
            jaccard_values[name] = jaccard_sim
        
        return  StringSimilarity.rank_vectors(jaccard_values)
        
    
    @staticmethod
    def create_dataframe(dict1, dict2, dict3, dict4): 
        
        """
        Creates a DataFrame from four dictionaries of similarity scores by each method.

        Args:
            dict1, dict2, dict3 (dict): Dictionaries of similarity scores seperated by method.

        Returns:
            DataFrame: A DataFrame with the similarity scores from the three dictionaries.

        """

        
        df = pd.DataFrame([dict1,dict2, dict3, dict4])
        
        df = df.T # Transpose to have keys as rows
    
        df.columns = ["dot_product", "cosine", "Euclidean", "jaccard"]
        
        return df 
    
    def user_interaction2(self, text_type, method="all", export= "No"): 
        
        """
        Facilitates user interaction for comparing a new text with the document pool.

        Returns:
            DataFrame: A DataFrame showing the similarity scores of the new text with each document in the pool.
        Raises:
            TypeError:  If Input is not a string
            ValueError: If User input is empty. 
            ValueError: If input word not string or file
            ValueError: If the Input string is longer than 200 characters. 
            ValueError: If User input is empty.

        """
        
        
        if not isinstance(text_type, str):
            raise TypeError("The text_type must be a string.")
        
        if text_type.strip() == "":
            raise ValueError("text_txpe is empty or only contains whitespace. Please enter valid text as argument.")
        
        if not text_type in ["string", "file"]: 
            raise ValueError("argument needs to be either string or file")
        
        
        if text_type == "string": 
            
            
        # Prompt the user to enter text
            q1 = input('Enter a string under 200 characters')
            
            if len(q1) > 500: 
                raise ValueError("Your input text was too long!")
            
            if q1.strip() == "":
                raise ValueError("Entered text is empty or only contains whitespace. Please enter valid text.")
            
            
            
        if text_type == "file": 
                    # Prompt the user to enter text
            q1 = input('Enter the name of the document (needs to be in the same directory as the script)')
            
            
            if q1.strip() == "":
                raise ValueError("Entered text is empty or only contains whitespace. Please enter valid text.")
            
            
            #check if file is in directory 
            
            objects = os.listdir(os.getcwd())
            
            if not q1 in objects: 
                
                raise ValueError(f"The file {q1} is not in the directory of this jupyter notebook.")
            
            
            
            try: 
                q1 = self.load_text(q1)
                
    
            except Exception as e: 
                            
                raise Exception(f'Failed to load document {q1} because of {e}')
            
        
        # cleans new text and create vector
        clean_text = self.main_cleaning(q1)
        new_vector = self.create_vector(clean_text)
        set_new_words = set(clean_text)
        
        
        try: 
        # Compute similarity scores
            result1 = self.dot_product_normal(clean_text, new_vector)
            result2 = self.cosine_Similarity(clean_text,new_vector)
            result3 = self.Euclidean_distance(clean_text, new_vector)
            result4 = self.Jaccard_similarity(clean_text, set_new_words)
            
        
            
            # Create and return a DataFrame with the results  
            if method == "all":   
                final_df = StringSimilarity.create_dataframe(result1, result2, result3, result4)
            
            elif method == "dot": 
                final_df =  pd.DataFrame(list(result1.items()), columns=['Document', 'Dot Product Similarity'])
            
            elif method == "cosine": 
                final_df =  pd.DataFrame(list(result2.items()), columns=['Document', 'Cosine Similarity'])
            
            elif method == "euclidean": 
                final_df =  pd.DataFrame(list(result3.items()), columns=['Document', 'Euclidean Distance'])
            
            elif method == "jaccard": 
                final_df = pd.DataFrame(list(result4.items()), columns=['Document', 'Jaccard Similarity'])
            
        
        except Exception as e:
            
            raise Exception(f"An error occurred while calculating similarity scores: {e}") 
    
        if export == "No": 
            return final_df 
        
        if export == "Yes": 
            try: 
                final_df.to_excel("results.xlsx")
                
                print("Excel has been created!")
                
                return final_df
                
            except Exception as e: 
                
                raise Exception(f"An Error occoured by creating Excel {e}") 
                
    def user_interaction(self, text_type, method="all", export="No"):
        """
        Facilitates user interaction for comparing a new text with the document pool.
        
        This function allows the user to input a text string or specify a file. It then performs text cleaning,
        computes various similarity scores with the documents in the pool, and optionally exports the results to Excel.

        Args:
            text_type (str): Type of text input, either "string" or "file".
            method (str, optional): The method to use for computing similarity scores. Defaults to "all".
            export (str, optional): Option to export the results to an Excel file. Defaults to "No".

        Returns:
            DataFrame: A DataFrame showing the similarity scores of the new text with each document in the pool.

        Raises:
            TypeError: If text_type is not a string.
            ValueError: If export is empty or not 'Yes'/'No'.
            ValueError: If text_type is empty or not 'string'/'file'.
            ValueError: If the input text is too long (more than 200 characters) or empty.
            ValueError: If the specified file is not found in the directory.
            Exception: For errors in loading the document or in similarity score calculations.
            Exception: For errors encountered while creating the Excel file.
        """
        
        # Check if text_type is a valid string
        if not isinstance(text_type, str):
            raise TypeError("The text_type must be a string.")
        
        # Check for empty string
        if text_type.strip() == "":
            raise ValueError("text_type is empty or only contains whitespace. Please enter valid text as argument.")
        
        # Validate text_type value
        if not export in ["Yes", "No"]: 
            raise ValueError("Argument needs to be either 'Yes' or 'No'.")
        
        if not text_type in ["string", "file"]: 
            raise ValueError("Argument needs to be either 'string' or 'file'.")

        # Handling 'string' input
        if text_type == "string": 
            # Prompt the user to enter text
            q1 = input('Enter a string under 500 characters')
            
            # Check for length constraint
            if len(q1) < 500: 
                raise ValueError("Your input text was too long!")
            
            # Check for empty input
            if q1.strip() == "":
                raise ValueError("Entered text is empty or only contains whitespace. Please enter valid text.")
            
        # Handling 'file' input
        if text_type == "file": 
            # Prompt the user to enter file name
            q1 = input('Enter the name of the document (needs to be in the same directory as the script)')
            
            # Check for empty input
            if q1.strip() == "":
                raise ValueError("Entered text is empty or only contains whitespace. Please enter valid text.")
            
            # Check if file exists in directory
            objects = os.listdir(os.getcwd())
            if not q1 in objects: 
                raise ValueError(f"The file {q1} is not in the directory of this Jupyter notebook.")
            
            # Attempt to load the text from the file
            try: 
                q1 = self.load_text(q1)
            except Exception as e: 
                raise Exception(f'Failed to load document {q1} because of {e}')
        
        # Clean the text and create vector
        clean_text = self.main_cleaning(q1)
        new_vector = self.create_vector(clean_text)
        set_new_words = set(clean_text)

        # Compute and store similarity scores
        try: 
            # Compute similarity scores
            result1 = self.dot_product_normal(clean_text, new_vector)
            result2 = self.cosine_Similarity(clean_text, new_vector)
            result3 = self.Euclidean_distance(clean_text, new_vector)
            result4 = self.Jaccard_similarity(clean_text, set_new_words)

            # Generate DataFrame based on selected method
            if method == "all":   
                final_df = StringSimilarity.create_dataframe(result1, result2, result3, result4)
            elif method == "dot": 
                final_df = pd.DataFrame(list(result1.items()), columns=['Document', 'Dot Product Similarity'])
            elif method == "cosine": 
                final_df = pd.DataFrame(list(result2.items()), columns=['Document', 'Cosine Similarity'])
            elif method == "euclidean": 
                final_df = pd.DataFrame(list(result3.items()), columns=['Document', 'Euclidean Distance'])
            elif method == "jaccard": 
                final_df = pd.DataFrame(list(result4.items()), columns=['Document', 'Jaccard Similarity'])
        except Exception as e:
            raise Exception(f"An error occurred while calculating similarity scores: {e}") 

        # Handle export option
        if export == "No": 
            return final_df 
        
        if export == "Yes": 
            try: 
                
                #export dataframe to results.xlsx
                final_df.to_excel("results.xlsx")
                print("Excel has been created!")
                return final_df
            except Exception as e: 
                raise Exception(f"An error occurred while creating Excel: {e}")           
        
            
            

In [38]:
#Class Document_similarity is set up 
Document_similarity = StringSimilarity()

#this line creates the document pool. It will read all files in the Corpus folder and create vectors for each 
Document_similarity.create_corpus()

#displays the document names of the pool
Document_similarity.document_pool.keys()

All vectors are updated


dict_keys(['text1.txt', 'text2.txt', 'text3.txt'])

In [3]:
#All text data will be processed and cleaned with the main_cleaning function 

text = "THIS is @n example string!!!stop-words, et 1, , 4 ,5 will be filtert__ out"

Document_similarity.main_cleaning(text)

['example', 'string', 'stopwords', 'filtert']

In [39]:
#it is possible to add texts until 500 characters as string to the document pool. 
#If the string contains vocabularies which were not part of the class dictionary they will be added to the pool and all vectors will be updated 

test_string = "this is a test"
Document_similarity.add_documents("dummy", test_string)

Text has been added to the pool but no new vocabularies were added


In [20]:
#For each document a vector will be created with the lenght of the class dictionary 
Document_similarity.vector_pool["dummy"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [62]:
print(Document_similarity.document_pool.keys())
print(len(Document_similarity.vector_pool["text1.txt"]))

new_text = "This text will be added to the document Pool because the script says so "

Document_similarity.add_documents("new_doc1",new_text)

print(Document_similarity.document_pool.keys())
print(len(Document_similarity.vector_pool["text1.txt"]))

dict_keys(['text1.txt', 'text2.txt', 'text3.txt', 'new_doc1'])
13396
All vectors are updated
dict_keys(['text1.txt', 'text2.txt', 'text3.txt', 'new_doc1'])
13396


In [29]:
Document_similarity.user_interaction("file", "dot")

,Document,Dot Product Similarity
0,text2.txt,3293
1,text1.txt,2594
2,text3.txt,1614


In [60]:
Document_similarity.user_interaction("file", "cosine")

,Document,Cosine Similarity
0,text2.txt,0.550761
1,text1.txt,0.519280
2,text3.txt,0.402100
3,new_doc1,0.019062


In [61]:
Document_similarity.user_interaction("file", "euclidean")

,Document,Euclidean Distance
0,text2.txt,78.752778
1,text1.txt,70.604532
2,text3.txt,69.303680
3,new_doc1,64.249514


In [63]:
Document_similarity.user_interaction("file", "jaccard")

,Document,Jaccard Similarity
0,text1.txt,0.234624
1,text2.txt,0.253854
2,text3.txt,0.163129
3,new_doc1,0.000394


In [36]:
Document_similarity.user_interaction("file", export="Yes")

Excel has been created!


,dot_product,cosine,Euclidean,jaccard
text2.txt,3293.0,0.550761,78.752778,0.253854
text1.txt,2594.0,0.519280,70.604532,0.234624
text3.txt,1614.0,0.402100,69.303680,0.163129
